In [37]:
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn 
patch_sklearn()
from sklearn.svm import SVC
from itertools import combinations
from imports import *

stroke_df = pd.read_csv("C:/Users/kfroo/OneDrive/Desktop/Thesis/Stroke Prediction Dataset/healthcare-dataset-stroke-data.csv")

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [38]:
stroke_df

#Drop null columns 
print(stroke_df.shape)
data = stroke_df.dropna()
data= data.drop('id', axis = 1)
print(data.shape)
#find categorical variables 
data_ohe = pd.get_dummies(data)
#One hot encode variables 
dummy_col_names = data_ohe.columns.to_list
#Keep list of encodings 


(5110, 12)
(4909, 11)


In [39]:
target_variable = 'smoking_status_smokes'

#data['Classification'] = data['Classification'].map({1:0, 2:1})

def preprocess(data):
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1:]#.squeeze()
    feature_names = X.columns
    scaler = StandardScaler()
    X = scaler.fit_transform(data.drop(columns = [target_variable], axis = 1))
    X = pd.DataFrame(X, columns = feature_names)
    y = y.reset_index().drop('index', axis = 1)
    data = pd.concat([X,y] ,axis = 1)
    x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=390, test_size=0.3)
    return x_train, x_test, y_train, y_test, feature_names

x_train, x_test, y_train, y_test, feature_names = preprocess(data_ohe)
train_data = pd.concat([x_train, y_train], axis = 1)
test_data = pd.concat([x_test, y_test], axis = 1)

In [40]:
train_data

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
646,-0.614796,-0.318067,-0.228208,-0.311914,-0.597616,-0.210875,0.833374,-0.833023,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,1.014569,-1.014569,-0.657926,-0.453376,1.284775,0
1545,1.158819,-0.318067,4.381968,2.512276,0.115463,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,1.014569,-1.014569,-0.657926,2.205673,-0.778346,0
3217,0.892777,-0.318067,-0.228208,0.004837,1.146881,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,-0.985640,0.985640,-0.657926,2.205673,-0.778346,0
822,-0.260073,-0.318067,-0.228208,0.023522,0.102730,-0.210875,0.833374,-0.833023,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,-0.985640,0.985640,-0.657926,-0.453376,1.284775,0
3915,0.538054,-0.318067,-0.228208,1.096917,0.344667,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,-1.157518,-0.432978,-0.397906,-0.985640,0.985640,-0.657926,-0.453376,-0.778346,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4567,0.183331,-0.318067,-0.228208,0.238741,-0.852287,-0.210875,0.833374,-0.833023,-0.014274,-0.729484,...,-0.067095,-1.157518,2.309587,-0.397906,1.014569,-1.014569,-0.657926,2.205673,-0.778346,0
2075,0.582394,-0.318067,-0.228208,-0.188545,-0.495747,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,-0.985640,0.985640,-0.657926,-0.453376,-0.778346,1
4111,-1.412922,-0.318067,-0.228208,-0.234471,-0.177409,-0.210875,0.833374,-0.833023,-0.014274,1.370831,...,-0.067095,-1.157518,-0.432978,2.513155,-0.985640,0.985640,-0.657926,-0.453376,1.284775,0
1266,0.892777,-0.318067,-0.228208,-0.618759,0.179131,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,1.014569,-1.014569,1.519928,-0.453376,-0.778346,0


In [41]:
def calculate_gini(y):
    """Calculate the Gini index for a given array of labels."""
    p = y.value_counts() / len(y)
    gini = 1 - np.sum(p ** 2)
    return gini
    
def gini_index(decision_function, y):
    """Calculate the Gini index for a given decision function and labels."""
    indices_less_zero = np.where(decision_function < 0)[0]
    indices_greater_zero = np.where(decision_function > 0)[0]

    g1 = y.iloc[indices_less_zero]
    g2 = y.iloc[indices_greater_zero]

    gini1 = calculate_gini(g1)
    gini2 = calculate_gini(g2)

    n = len(y)
    gini_score = (len(g1) / n) * gini1 + (len(g2) / n) * gini2

    return gini_score

In [42]:
def test_split(decision_function, data):
    """Split the data into left and right data frames based on the decision function."""
    indices_less_zero = np.where(decision_function < 0)[0]
    indices_greater_zero = np.where(decision_function > 0)[0]

    left_data = data.iloc[indices_less_zero]
    right_data = data.iloc[indices_greater_zero]
    
    return left_data, right_data
    #return {'left':left_data, 'right':right_data}

In [43]:
def get_split(data):
    """Find the best Gini split for the given data."""
    best_gini = float('inf')
    best_split = None

    features = data.iloc[:, :-1]
    target = data.iloc[:, -1]

    #Generate all possible combinations of feature pairs
    feature_pairs = list(combinations(features.columns, 2))
    #decision_functions_array = []

    for pair in feature_pairs:
    #    # Select the pair of features
        feature1, feature2 = pair
        selected_features = features[[feature1, feature2]]
        #print(selected_features)

        # Train the SVM
        svm = SVC(kernel='linear', C = 100)
        svm.fit(selected_features, target)

        # Use the decision function to split the data
        decision_function = svm.decision_function(selected_features)
    
        gini = gini_index(decision_function, target)
        #all_gini.append(gini)
 
        if gini < best_gini:
            best_gini = gini
            best_df = decision_function
            best_svm = svm
            best_split = {'feat1': selected_features.columns[0], 
                          'feat2': selected_features.columns[1],
                          'x1': np.around((best_svm.coef_[0][0]), 5), 
                          'x2': np.around((best_svm.coef_[0][1]), 5),
                          'intercept': np.around(best_svm.intercept_[0], 5)}
            rule = f"{best_split['x1']}*{best_split['feat1']} + {best_split['x2']}*{best_split['feat2']} + {best_split['intercept']} < 0"
            #f"{np.around((best_svm.coef_[0][0]), 5)}*{selected_features.columns[0]} + {np.around((best_svm.coef_[0][1]), 5)}*{selected_features.columns[1]} + {np.around(best_svm.intercept_[0], 5)} < 0 "
        
        groups = test_split(best_df, data)

    return {'best_split':best_split, 'split_rule': rule, 'best_df':best_df, 'best_svm':best_svm, 'groups':groups}

In [44]:
# Create a terminal node value
def to_terminal(subset):
    outcomes = list((subset.iloc[:,-1]))
    return max(set(outcomes), key=outcomes.count)

In [45]:
def split(node, max_depth, min_samples, depth):
    left, right = node['groups']
    del(node['groups'])

    #Check for no-split
    if left.empty or right.empty:
        node['left']  = node['right'] = to_terminal(pd.concat([left, right]))
        return
    
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_samples:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_samples, depth+1)

     # process right child
    if len(right) <= min_samples:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_samples, depth+1)


In [46]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
   root = get_split(train)
   split(root, max_depth, min_size, 0)
   return root

In [47]:
def predict(instance, node):
    if 'split_rule' not in node:
        return node['left']  # Terminal node value
    split_rule = node['split_rule']
    print(split_rule)

In [48]:
# Print a decision tree
def print_tree(node, depth=0):
 if isinstance(node, dict):
    print(depth*'  ', f'Feature:{node["split_rule"]}')
    #print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
    print_tree(node['left'], depth+1)
    print_tree(node['right'], depth+1)
 else:
   print('%s[%s]' % ((depth*'  ', node)))

In [49]:
# Make a prediction with a decision tree
def predict(node, row):
    feat1 = node['best_split']['feat1']
    feat2 = node['best_split']['feat2']
    x1 = node['best_split']['x1']
    x2 = node['best_split']['x2']
    intercept = node['best_split']['intercept']

    score = x1 * row[feat1] + x2 * row[feat2] + intercept 
    if score < 0:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [50]:
train_data

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
646,-0.614796,-0.318067,-0.228208,-0.311914,-0.597616,-0.210875,0.833374,-0.833023,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,1.014569,-1.014569,-0.657926,-0.453376,1.284775,0
1545,1.158819,-0.318067,4.381968,2.512276,0.115463,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,1.014569,-1.014569,-0.657926,2.205673,-0.778346,0
3217,0.892777,-0.318067,-0.228208,0.004837,1.146881,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,-0.985640,0.985640,-0.657926,2.205673,-0.778346,0
822,-0.260073,-0.318067,-0.228208,0.023522,0.102730,-0.210875,0.833374,-0.833023,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,-0.985640,0.985640,-0.657926,-0.453376,1.284775,0
3915,0.538054,-0.318067,-0.228208,1.096917,0.344667,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,-1.157518,-0.432978,-0.397906,-0.985640,0.985640,-0.657926,-0.453376,-0.778346,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4567,0.183331,-0.318067,-0.228208,0.238741,-0.852287,-0.210875,0.833374,-0.833023,-0.014274,-0.729484,...,-0.067095,-1.157518,2.309587,-0.397906,1.014569,-1.014569,-0.657926,2.205673,-0.778346,0
2075,0.582394,-0.318067,-0.228208,-0.188545,-0.495747,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,-0.985640,0.985640,-0.657926,-0.453376,-0.778346,1
4111,-1.412922,-0.318067,-0.228208,-0.234471,-0.177409,-0.210875,0.833374,-0.833023,-0.014274,1.370831,...,-0.067095,-1.157518,-0.432978,2.513155,-0.985640,0.985640,-0.657926,-0.453376,1.284775,0
1266,0.892777,-0.318067,-0.228208,-0.618759,0.179131,-0.210875,-1.199942,1.200447,-0.014274,-0.729484,...,-0.067095,0.863918,-0.432978,-0.397906,1.014569,-1.014569,1.519928,-0.453376,-0.778346,0


In [51]:
train_data.smoking_status_smokes.value_counts()

0    2941
1     495
Name: smoking_status_smokes, dtype: int64

In [53]:
tree = build_tree(train_data, 4, 20)
print_tree(tree)

In [ ]:
y_true = list(test_data[target_variable])

In [ ]:
predictions_saved = []
for row in range(len(test_data)):
    #print(test_data.iloc[row])
    p = predict(tree, test_data.iloc[row])
    predictions_saved.append(p)
y_pred = list(predictions_saved)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

0.8488345779588142